In [1]:
!pip install -U torch transformers tokenizers accelerate safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 7.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.8 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 4.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 74.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 50.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 59.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 7.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━

# KoAlpaca

In [3]:
import torch
from transformers import pipeline, AutoModelForCausalLM

MODEL = 'beomi/KoAlpaca-Polyglot-5.8B'

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)
model.eval()

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30080, 4096)
    (layers): ModuleList(
      (0-27): 28 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (embed_out): Linear(in_features=4096, out_features=30080, bias=False)
)

In [5]:
pipe = pipeline(
    'text-generation', 
    model=model,
    tokenizer=MODEL,
    device=0
)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [6]:
def ask(x, context='', is_input_full=False):
    ans = pipe(
        f"### 질문: {x}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {x}\n\n### 답변:", 
        do_sample=True, 
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )
    print(ans[0]['generated_text'])

In [7]:
ask("2023년 대한민국의 대통령은 누구야")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2023년 대한민국의 대통령은 조 바이든입니다.


In [11]:
ask("2023년 미국 대통령은 누구야")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2023년 미국 대통령은 조 바이든입니다.


In [12]:
ask("2023년 대한민국 대통령은 누구야")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2023년 대한민국 대통령은 1대 대통령인 이승만 대통령입니다. 이승만 대통령은 1899년 12월 20일에 취임했고, 1960년 4월 27일에는 대한민국 최초의 대통령으로 취임했습니다. 그 후 1949년에는 부통령이 되었다가 1960년에 대한민국의 첫 대통령으로 취임했습니다. 이승만 대통령은 대한민국 역사상 가장 높은 지위에 있었으며, 대통령으로 활동했던 기간은 1947년부터 1960년까지였습니다. 


In [8]:
ask("KoAlpaca에 대해 설명해줘")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Koalpaca는 Alpha Central Africana의 준말로, 초기의 발견자 John H. Freed가 이 대륙에서 최초로 발견한 포유류입니다. 이 동물은 특정한 시기에 대해서는 남미와 북미, 아프리카를 아우르는 지역에 서식했던 것으로 알려져 있습니다. 코알라와 비슷한 특징을 가지고 있지만, 코알라는 남극 지역에서 발견되는 반면, Koalpaca는 열대 지역에서 발견됩니다. 이 동물은 가장 높은 나무의 잎사귀를 먹고, 하루에 20시간 이상 잠을 자기도 하며, 특이한 울음소리를 내는 동물로 유명합니다. 전체적으로 보아 이 동물은 코알라와 비슷한 특징을 가지고 있지만, 더 많은 시간을 활동하며 울기도 하기 때문에, 좀 더 아프리카에 가까운 동물이라고 할 수 있습니다. 


In [9]:
ask("딸기에 대해 설명해줘")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


딸기는 쌍떡잎식물 이판화군 장미목 딸기과의 한해살이풀로, 높이는 10~30cm 정도이며 열매는 장각이라는 달리는 감으로 재배됩니다. 딸기의 원산지는 중국이며, 한국에는 조선시대에 관련 기록과 전주감영에서 재배가 기록되어 있습니다. 딸기는 5월에 많이 수확되며, 이삭이 나오고, 열매가 맺히며 씨를 통해 번식이 가능합니다. 딸기에는 자양강장 효과가 있는 비타민 C와 B, 그리고 항산화 물질이 포함된 리코펜 등이 함유되어 있어, 기운을 북돋우고, 암을 예방하는 효과가 있습니다. 하지만 딸기의 경우, 수확 후에는 당분이 많아져서 산미가 약해지며, 저장력이 감소하기 때문에 수확 후 3일 이내에 먹는 것이 좋습니다. 


# KULLM

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

#from prompter import Prompter

In [3]:
MODEL = "nlpai-lab/kullm-polyglot-5.8b-v2"

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)
#model.eval()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=MODEL, device=0)
prompter = Prompter("kullm")

In [ ]:
def infer(instruction="", input_text=""):
    prompt = prompter.generate_prompt(instruction, input_text)
    output = pipe(prompt, max_length=512, temperature=0.2, num_beams=5, eos_token_id=2)
    s = output[0]["generated_text"]
    result = prompter.get_response(s)

    return print(result)

In [7]:
infer(input_text="고려대학교에 대해서 알려줘")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


고려대학교는 한국에서 가장 오래되고 가장 큰 대학 중 하나입니다. 한국에서 가장 오래된 대학 중 하나이며 한국에서 가장 큰 대학 중 하나입니다. 고려대학교는 한국에서 가장 오래된 대학 중 하나이며 한국에서 가장 큰 대학 중 하나입니다. 고려대학교는 한국에서 가장 오래된 대학 중 하나이며 한국에서 가장 큰 대학 중 하나입니다. 고려대학교는 한국에서 가장 오래된 대학 중 하나이며 한국에서 가장 큰 대학 중 하나입니다. 고려대학교는 한국에서 가장 오래된 대학 중 하나이며 한국에서 가장 큰 대학 중 하나입니다. 고려대학교는 한국에서 가장 오래된 대학 중 하나이며 한국에서 가장 큰 대학 중 하나입니다. 고려대학교는 한국에서 가장 오래된 대학 중 하나이며 한국에서 가장 큰 대학 중 하나입니다. 고려대학교는 한국에서 가장 오래된 대학 중 하나이며 한국에서 가장 큰 대학 중 하나입니다. 고려대학교는 한국에서 가장 오래된 대학 중 하나이며 한국에서 가장 큰 대학 중 하나입니다. 고려대학교는 한국에서 가장 오래된 대학 중 하나이며 한국에서 가장 큰 대학 중 하나입니다. 고려대학교는 한국에서 가장 오래된 대학 중 하나이며 한국에서 가장 큰 대학 중 하나입니다. 고려대학교는 한국에서 가장 오래된 대학 중 하나이며 한국에서 가장 큰 대학 중 하나입니다. 고려대학교는 한국에서 가장 오래된 대학 중 하나이며 한국에서 가장 큰 대학 중 하나입니다. 고려대학교는 한국에서 가장 오래된 대학 중 하나이며 한국에서 가장 큰 대학 중 하나입니다. 고려대학교는 한국에서 가장 오래된 대학 중 하나이며 한국에서 가장 큰 대학 중 하나입니다. 고려대학교는 한국에서 가장 오래된 대학 중 하나이며 한국에서 가장 큰 대학 중 하나입니다. 고려대학교는 한국에서 가장 오래된 대학 중 하나이며 한국에서 가장 큰 대학 중 하나입니다. 고려대학교는 한국에서 가장 오래된 대학 중 하나이며 한국에서 가장 큰 대학 중 하나입니다. 고려대학교는 한국에서 가장 오래된 대학 중 하나이며 한국에서 가장 큰 대학 중 하나입니다. 고

In [8]:
infer(input_text="2023년 대한민국의 대통령은 누구야")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2023년 대한민국의 대통령은 문재인 대통령입니다.


In [9]:
infer(input_text="2023년 대한민국 대통령은 누구야")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2023년 대한민국 대통령은 문재인 대통령입니다.


In [10]:
infer(input_text="2023년 미국 대통령은 누구야")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2023년 미국 대통령은 조 바이든입니다.


In [12]:
infer(input_text="딸기에 대해 설명해줘")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다:

딸기에 대한 설명은 다음과 같습니다


# fine-tuning

In [4]:
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/huggingface/peft.git

!pip install openai
!pip install scipy
!pip install spacy-transformers

!pip install bitsandbytes
!python -m bitsandbytes
!pip install protobuf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 3.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 6.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 6.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.3/913.3 kB 8.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 7.9

In [5]:
import os

import torch
import bitsandbytes
from datasets import load_dataset
import transformers
from transformers import LlamaTokenizer, AutoTokenizer, AutoConfig, GenerationConfig, AutoModelForCausalLM

from peft import prepare_model_for_kbit_training


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda116.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda116.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/lib/x86_64-linux-gnu did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('noninteractive    SHELL=/bin/bash')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//

In [6]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:          1.0Ti       131Gi       229Gi        18Gi       647Gi       851Gi
Swap:            0B          0B          0B


In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [8]:
model.train()

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30080, 4096)
    (layers): ModuleList(
      (0-27): 28 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (embed_out): Linear(in_features=4096, out_features=30080, bias=False)
)

In [8]:
kullm_model = model

In [9]:
model_1 = prepare_model_for_kbit_training(kullm_model)

In [10]:
from peft import LoraConfig, get_peft_model, TaskType

# PEFT 설정
config = LoraConfig(
    r=8,    # Lora attention dimension / LoRA를 적용하는 가중치 행렬의 수(차원)
    lora_alpha=16, # 로라 스케일링 값
    target_modules=["query_key_value", "xxx"], # LoRA를 적용할 모듈의 이름
    lora_dropout=0.05, # dropout 값
    bias="none",
    task_type="CAUSAL_LM"
)

In [11]:
model = get_peft_model(model_1, config)
model.print_trainable_parameters()

trainable params: 3,670,016 || all params: 5,888,729,088 || trainable%: 0.06232271760435925


### 597 data

In [24]:
data = load_dataset("json", data_files="/workspace/data/kullm_koen_no_arrow_test.json")
data

Found cached dataset json (/root/.cache/huggingface/datasets/json/default-3a5a69d39745ed8f/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 597
    })
})

In [25]:
def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["instruction"]:
        return f"""
        ### Instruction:
        {data_point["instruction"]}

        ### Input:
        {data_point["input"]}

        ### Response:
        {data_point["output"]}"""

In [26]:
# pad 값 지정
tokenizer.pad_token_id = 0

# map(함수, 리스트) -> map : 리스트로부터 원소를 하나씩 꺼내 함수를 적용시킨 다음, 그 결과를 새로운 리스트에 담아준다. 
# 데이터 섞은 다음 lambda식 사용해서 tokenizer 사용 후 다시 리스트에 담아주는 것
data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=True,
        max_length=256,
        padding="max_length",
    )
)
data

Map:   0%|          | 0/597 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 597
    })
})

In [27]:
MICRO_BATCH_SIZE = 4
BATCH_SIZE = 128
EPOCHS = 19
LEARNING_RATE = 2e-5

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=3,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True
        logging_steps=1,
        output_dir="lora-lama",
        save_total_limit=3,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [28]:
model.config.use_cache = False

In [29]:
trainer.train()

Step,Training Loss
1,2.174100
2,2.172100
3,2.144000
4,2.123700
5,2.122400
6,2.090600
7,2.050900
8,2.076400
9,1.997500
10,2.005200


RuntimeError: [enforce fail at inline_container.cc:337] . unexpected pos 19129135872 vs 19129135768

In [20]:
# torch.save(obj=model, f='/workspace/KULLM_FT.pt')

In [30]:
generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
)

In [31]:
word_1 = "Please enter a text message" # typingText
word_2 = "아래로 스크롤해줘" #scrolldown
word_3 = "링크를 열어서 웹 주소로 이동" # getURL
word_4 = "Connect to the address" # getURL
word_5 = "브라우저를 열어줘" # openBrowser
word_6 = "입력 글자를 타이핑 해주세요" # typingText

In [32]:
def prompt(word) :
    return f"""
        ### Instruction:
        다음은 요청에 적절하게 응답하는 알파카 작업 스크립트입니다.
        
        ### Input:
        {word}

        ### Response:
        """

In [33]:
def answer(inputs) :
    input_ids = inputs["input_ids"].cuda()

    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=50,
        pad_token_id=tokenizer.eos_token_id,
    )
    for s in generation_output.sequences:
        print(tokenizer.decode(s))

In [34]:
inputs = tokenizer(prompt(word_1), return_tensors="pt")
answer(inputs)  # typingText


        ### Instruction:
        다음은 요청에 적절하게 응답하는 알파카 작업 스크립트입니다.
        
        ### Input:
        Please enter a text message

        ### Response:
        openBrowserReplyButton

        ### Browse your web browser
        scrollD


In [35]:
inputs = tokenizer(prompt(word_2), return_tensors="pt")
answer(inputs) #scrolldown


        ### Instruction:
        다음은 요청에 적절하게 응답하는 알파카 작업 스크립트입니다.
        
        ### Input:
        아래로 스크롤해줘

        ### Response:
        openBrowserReader(인터페이스: GoogleChromeWebStormScreen, Parameters:            TextIn


In [36]:
inputs = tokenizer(prompt(word_3), return_tensors="pt")
answer(inputs) # getURL


        ### Instruction:
        다음은 요청에 적절하게 응답하는 알파카 작업 스크립트입니다.
        
        ### Input:
        링크를 열어서 웹 주소로 이동

        ### Response:
        openBrowserRedirectUrl("www.example.com/browser")

        ### GetManageByTar


In [37]:
inputs = tokenizer(prompt(word_4), return_tensors="pt")
answer(inputs) # getURL


        ### Instruction:
        다음은 요청에 적절하게 응답하는 알파카 작업 스크립트입니다.
        
        ### Input:
        Connect to the address

        ### Response:
        openBrowserReplyButton

        ### Usage:
        scrollDownPage

  


In [38]:
inputs = tokenizer(prompt(word_5), return_tensors="pt")
answer(inputs) # openBrowser


        ### Instruction:
        다음은 요청에 적절하게 응답하는 알파카 작업 스크립트입니다.
        
        ### Input:
        브라우저를 열어줘

        ### Response:
        openBrowser

        ### Login and Settings
        setGuestLogin

     


In [39]:
inputs = tokenizer(prompt(word_6), return_tensors="pt")
answer(inputs) # typingText


        ### Instruction:
        다음은 요청에 적절하게 응답하는 알파카 작업 스크립트입니다.
        
        ### Input:
        입력 글자를 타이핑 해주세요

        ### Response:
        openBrowser

        ### Result:
        typingText

        scrol


## 1000 data

In [15]:
data = load_dataset("json", data_files="/workspace/data/kullm_koen_no_arrow_test_1000over.json")
data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-b3377b1fcba746e5/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 1000
    })
})

In [16]:
def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["instruction"]:
        return f"""
        ### Instruction:
        {data_point["instruction"]}

        ### Input:
        {data_point["input"]}

        ### Response:
        {data_point["output"]}"""

In [17]:
# pad 값 지정
tokenizer.pad_token_id = 0

# map(함수, 리스트) -> map : 리스트로부터 원소를 하나씩 꺼내 함수를 적용시킨 다음, 그 결과를 새로운 리스트에 담아준다. 
# 데이터 섞은 다음 lambda식 사용해서 tokenizer 사용 후 다시 리스트에 담아주는 것
data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=True,
        max_length=256,
        padding="max_length",
    )
)
data

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [20]:
MICRO_BATCH_SIZE = 4
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 2e-5

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,   # 학습을 위한 GPU/TPU 코어/CPU당 배치 크기. 기본값 8
        gradient_accumulation_steps=3, # 업데이트 스텝 수(GRADIENT_ACCUMULATION_STEPS). 기본값은 1. TrainingArguments 예시에 적용되어있는 값 사용
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,  #  Apex AMP 최적화 수준?
        logging_steps=1, # 두 로그 사이의 업데이트 단계 수? 0 혹은 1이면 단계의 비율로 해석된다는 듯. 
        output_dir="lora-lama",
        save_total_limit=3, # 체크포인트의 총량을 제한한다. 수를 넘으면 이전의 체크포인트를 삭제한다.
    ),

    # data_collator : 데이터 집합 요소 목록을 입력으로 사용하여 배치를 형성하는 개체
    # DataCollatorForLanguageModeling : 언어 모델링에 사용되는 데이터 수집기. 입력이 모두 동일한 크기가 아닌경우 배치의 최대 길이까지 알아서 padding된다.
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [21]:
model.config.use_cache = False
trainer.train()

Step,Training Loss
1,1.346900
2,1.337600
3,1.315500
4,1.387500
5,1.325500
6,1.310800
7,1.205500
8,1.198600
9,1.175700
10,1.170300


TrainOutput(global_step=830, training_loss=0.18052395470889218, metrics={'train_runtime': 590.629, 'train_samples_per_second': 16.931, 'train_steps_per_second': 1.405, 'total_flos': 8.820417519157248e+16, 'train_loss': 0.18052395470889218, 'epoch': 9.96})

In [22]:
generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
)

In [23]:
word_1 = "Please enter a text message" # typingText
word_2 = "아래로 스크롤해줘" #scrolldown
word_3 = "링크를 열어서 웹 주소로 이동" # getURL
word_4 = "Connect to the address" # getURL
word_5 = "브라우저를 열어줘" # openBrowser
word_6 = "입력 글자를 타이핑 해주세요" # typingText

In [24]:
def prompt(word) :
    return f"""
        ### Instruction:
        다음은 요청에 적절하게 응답하는 알파카 작업 스크립트입니다.
        
        ### Input:
        {word}

        ### Response:
        """

In [25]:
def answer(inputs) :
    input_ids = inputs["input_ids"].cuda()

    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=50,
        pad_token_id=tokenizer.eos_token_id,
    )
    for s in generation_output.sequences:
        print(tokenizer.decode(s))

In [26]:
inputs = tokenizer(prompt(word_1), return_tensors="pt")
answer(inputs)  # typingText


        ### Instruction:
        다음은 요청에 적절하게 응답하는 알파카 작업 스크립트입니다.
        
        ### Input:
        Please enter a text message

        ### Response:
        openBrowserReader(GetScriptData, TypeError);
        helperTestControll


In [27]:
inputs = tokenizer(prompt(word_2), return_tensors="pt")
answer(inputs) #scrolldown


        ### Instruction:
        다음은 요청에 적절하게 응답하는 알파카 작업 스크립트입니다.
        
        ### Input:
        아래로 스크롤해줘

        ### Response:
        characters = GetBodyControl()
        try
        rotateBackwardRight, downRight



In [28]:
inputs = tokenizer(prompt(word_3), return_tensors="pt")
answer(inputs) # getURL


        ### Instruction:
        다음은 요청에 적절하게 응답하는 알파카 작업 스크립트입니다.
        
        ### Input:
        링크를 열어서 웹 주소로 이동

        ### Response:
        openBrowserDocumentsRunningLinkURL
         ? GetURLNotFound
         


In [29]:
inputs = tokenizer(prompt(word_4), return_tensors="pt")
answer(inputs) # getURL


        ### Instruction:
        다음은 요청에 적절하게 응답하는 알파카 작업 스크립트입니다.
        
        ### Input:
        Connect to the address

        ### Response:
        openBrowserRequestURL := "https://content.google.com/messages/new?id=Gui%20Zoom


In [30]:
inputs = tokenizer(prompt(word_5), return_tensors="pt")
answer(inputs) # openBrowser


        ### Instruction:
        다음은 요청에 적절하게 응답하는 알파카 작업 스크립트입니다.
        
        ### Input:
        브라우저를 열어줘

        ### Response:
        openBrowserDown
          smtptransferRegisterFetchScript
          ty


In [31]:
inputs = tokenizer(prompt(word_6), return_tensors="pt")
answer(inputs) # typingText


        ### Instruction:
        다음은 요청에 적절하게 응답하는 알파카 작업 스크립트입니다.
        
        ### Input:
        입력 글자를 타이핑 해주세요

        ### Response:
        openBrowserRequestUnmatching
          rdownloadDownloadLinks
          cont


## 1000 data (instruction 영어)

In [12]:
data = load_dataset("json", data_files="/workspace/kullm_input_x_instruction_o_delimiter_A.json")
data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-21dbadf82163dbd2/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['output', 'instruction', 'input'],
        num_rows: 597
    })
})

In [13]:
def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["instruction"]:
        return f"""
        ### Instruction:
        {data_point["instruction"]}

        ### Input:
        {data_point["input"]}

        ### Response:
        {data_point["output"]}"""

In [14]:
# pad 값 지정
tokenizer.pad_token_id = 0

# map(함수, 리스트) -> map : 리스트로부터 원소를 하나씩 꺼내 함수를 적용시킨 다음, 그 결과를 새로운 리스트에 담아준다. 
# 데이터 섞은 다음 lambda식 사용해서 tokenizer 사용 후 다시 리스트에 담아주는 것
data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=True,
        max_length=256,
        padding="max_length",
    )
)
data

Map:   0%|          | 0/597 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['output', 'instruction', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 597
    })
})

In [29]:
MICRO_BATCH_SIZE = 4
BATCH_SIZE = 128
EPOCHS = 12
LEARNING_RATE = 3e-4

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,   # 학습을 위한 GPU/TPU 코어/CPU당 배치 크기. 기본값 8
        gradient_accumulation_steps=8, # 업데이트 스텝 수(GRADIENT_ACCUMULATION_STEPS). 기본값은 1. TrainingArguments 예시에 적용되어있는 값 사용
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,  #  Apex AMP 최적화 수준?
        logging_steps=1, # 두 로그 사이의 업데이트 단계 수? 0 혹은 1이면 단계의 비율로 해석된다는 듯. 
        output_dir="lora-lama",
        save_total_limit=3, # 체크포인트의 총량을 제한한다. 수를 넘으면 이전의 체크포인트를 삭제한다.
    ),

    # data_collator : 데이터 집합 요소 목록을 입력으로 사용하여 배치를 형성하는 개체
    # DataCollatorForLanguageModeling : 언어 모델링에 사용되는 데이터 수집기. 입력이 모두 동일한 크기가 아닌경우 배치의 최대 길이까지 알아서 padding된다.
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [30]:
model.config.use_cache = False
trainer.train()

Step,Training Loss
1,0.094300
2,0.378300
3,0.188200
4,0.165100
5,0.142800
6,0.142900
7,0.146100
8,0.138000
9,0.122600
10,0.118100


TrainOutput(global_step=216, training_loss=0.09183765333835725, metrics={'train_runtime': 386.8653, 'train_samples_per_second': 18.518, 'train_steps_per_second': 0.558, 'total_flos': 6.091932943201075e+16, 'train_loss': 0.09183765333835725, 'epoch': 11.52})

In [31]:
generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
)

In [32]:
word_1 = "Please enter a text message" # typingText
word_2 = "아래로 스크롤해줘" #scrolldown
word_3 = "링크를 열어서 웹 주소로 이동" # getURL
word_4 = "Connect to the address" # getURL
word_5 = "브라우저를 열어줘" # openBrowser
word_6 = "입력 글자를 타이핑 해주세요" # typingText

In [33]:
def prompt(word) :
    return f"""
        ### Instruction:
        This is an alpaca script
        
        ### Input:
        {word}

        ### Response:
        """

In [34]:
def answer(inputs) :
    input_ids = inputs["input_ids"].cuda()

    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=50,
        pad_token_id=tokenizer.eos_token_id,
    )
    for s in generation_output.sequences:
        print(tokenizer.decode(s))

In [35]:
inputs = tokenizer(prompt(word_1), return_tensors="pt")
answer(inputs)  # typingText


        ### Instruction:
        This is an alpaca script
        
        ### Input:
        Please enter a text message

        ### Response:
        openBrowser

        GetURL

        URL을 통해 접속

        GetText



In [36]:
inputs = tokenizer(prompt(word_2), return_tensors="pt")
answer(inputs) #scrolldown


        ### Instruction:
        This is an alpaca script
        
        ### Input:
        아래로 스크롤해줘

        ### Response:
        openBrowser

        ### Input:
        

        ### Response:



In [37]:
inputs = tokenizer(prompt(word_3), return_tensors="pt")
answer(inputs) # getURL


        ### Instruction:
        This is an alpaca script
        
        ### Input:
        링크를 열어서 웹 주소로 이동

        ### Response:
        openBrowser

        GetURL

        URL을 통해 접속

        GetURL



In [38]:
inputs = tokenizer(prompt(word_4), return_tensors="pt")
answer(inputs) # getURL


        ### Instruction:
        This is an alpaca script
        
        ### Input:
        Connect to the address

        ### Response:
        openBrowser

        GetURL

        URL을 통해 접속

        GetURL



In [39]:
inputs = tokenizer(prompt(word_5), return_tensors="pt")
answer(inputs) # openBrowser


        ### Instruction:
        This is an alpaca script
        
        ### Input:
        브라우저를 열어줘

        ### Response:
        openBrowser

        ### Input:
        

        ### Response:



In [40]:
inputs = tokenizer(prompt(word_6), return_tensors="pt")
answer(inputs) # typingText


        ### Instruction:
        This is an alpaca script
        
        ### Input:
        입력 글자를 타이핑 해주세요

        ### Response:
        openBrowser

        ### Input:
        

        ### Response:

